# 02 · Preparación de features (activos)


In [1]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))
from dotenv import load_dotenv
import os, pandas as pd, numpy as np

load_dotenv()

from src.utils import crear_kpis_pptos, unificar_cobertura_salud, marcar_activos

# Carpeta raíz del proyecto (sube un nivel desde /notebooks)
ROOT = Path.cwd().parent
DEFAULT = ROOT / "data" / "raw" / "Tab_Clientes(2).csv"

# Permite override con .env si algún día quieres mover la ruta
DATA_PATH = Path(os.getenv("DATA_PATH", str(DEFAULT)))

df = pd.read_csv(DATA_PATH, low_memory=False)
pd.set_option("display.max_columns", None)  # no truncar columnas
pd.set_option("display.width", None) 


In [ ]:
# 1. KPIs presupuestos
df_kpis = crear_kpis_pptos(df)

cols_preview = [
    'CantPptos','TotPptos','CantPptosAbo','TotPptosAbo','CantPptosAvan','TotPptosAvan',
    'TicketPromPpto','PctPptosAbonados','PctPptosAvanzados','PctCumplimiento','MontoAbonadoProm'
]
[df for df in cols_preview if df in df_kpis.columns], df_kpis[cols_preview].head(3)


(['CantPptos',
  'TotPptos',
  'CantPptosAbo',
  'TotPptosAbo',
  'CantPptosAvan',
  'TotPptosAvan',
  'TicketPromPpto',
  'PctPptosAbonados',
  'PctPptosAvanzados',
  'PctCumplimiento',
  'MontoAbonadoProm'],
    CantPptos  TotPptos  CantPptosAbo  TotPptosAbo  CantPptosAvan  \
 0        NaN       NaN           NaN          NaN            NaN   
 1        NaN       NaN           NaN          NaN            NaN   
 2        NaN       NaN           NaN          NaN            NaN   
 
    TotPptosAvan  TicketPromPpto  PctPptosAbonados  PctPptosAvanzados  \
 0           NaN             NaN               NaN                NaN   
 1           NaN             NaN               NaN                NaN   
 2           NaN             NaN               NaN                NaN   
 
    PctCumplimiento  MontoAbonadoProm  
 0              NaN               NaN  
 1              NaN               NaN  
 2              NaN               NaN  )

In [ ]:
# 1.2 Comprobación general
cols = ['CantPptos','TotPptos','CantPptosAbo','TotPptosAbo','CantPptosAvan','TotPptosAvan',
        'TicketPromPpto','PctPptosAbonados','PctPptosAvanzados','PctCumplimiento','MontoAbonadoProm']

# 1) ¿Cuántos tienen actividad de presupuestos?
print("Proporción con CantPptos notna:", df_kpis['CantPptos'].notna().mean())
print("Proporción con CantPptos > 0 :", (pd.to_numeric(df_kpis['CantPptos'], errors='coerce') > 0).mean())

# 2) Muestra SOLO pacientes con presupuesto (>0) para ver KPIs calculados
mask_act = pd.to_numeric(df_kpis['CantPptos'], errors='coerce') > 0
df_kpis.loc[mask_act, cols].sample(5, random_state=42)


Proporción con CantPptos notna: 0.47447783828496587
Proporción con CantPptos > 0 : 0.47447783828496587


,CantPptos,TotPptos,CantPptosAbo,TotPptosAbo,CantPptosAvan,TotPptosAvan,TicketPromPpto,PctPptosAbonados,PctPptosAvanzados,PctCumplimiento,MontoAbonadoProm
48635,2.0,686850.0,1.0,23700.0,1.0,23700.0,3.434250e+05,0.500000,0.500000,0.500000,23700.0
53099,3.0,679490.0,2.0,283990.0,2.0,262390.0,2.264967e+05,0.666667,0.666667,0.666667,141995.0
34335,1.0,396100.0,0.0,0.0,0.0,0.0,3.961000e+05,0.000000,0.000000,0.000000,NaN
36249,1.0,174900.0,0.0,0.0,0.0,0.0,1.749000e+05,0.000000,0.000000,0.000000,NaN
33221,1.0,1874100.0,0.0,0.0,0.0,0.0,1.874100e+06,0.000000,0.000000,0.000000,NaN


In [4]:
## 2. Transformación log1p de montos

# columnas de montos a transformar
montos = ['TotPptos', 'TotPptosAbo', 'TotPptosAvan']

df_m = df_kpis.copy()

# asegurar numérico y limpiar negativos (si los hay) -> NaN
for c in montos:
    if c in df_m.columns:
        df_m[c] = pd.to_numeric(df_m[c], errors='coerce')
        df_m.loc[df_m[c] < 0, c] = np.nan  # salvaguarda

# crear columnas transformadas con log1p
for c in montos:
    if c in df_m.columns:
        df_m[c + '_l1p'] = np.log1p(df_m[c])

# (opcional) transformar el ticket promedio si lo vas a usar en el modelo
if 'TicketPromPpto' in df_m.columns:
    df_m['TicketPromPpto_l1p'] = np.log1p(df_m['TicketPromPpto'])

# diagnóstico rápido: percentiles antes / después
def resumen_percentiles(serie):
    q = serie.quantile([0, .5, .9, .95, .99], interpolation='linear')
    q.index = ['min','p50','p90','p95','p99']
    return q

diag = {}
for c in montos:
    if c in df_m.columns and (c + '_l1p') in df_m.columns:
        diag[c] = pd.DataFrame({
            'original': resumen_percentiles(df_m[c].dropna()),
            'log1p': resumen_percentiles(df_m[c + '_l1p'].dropna())
        })

# mostrar resumen
for c, tabla in diag.items():
    print(f"\n==== {c} ====")
    display(tabla)

# vistazo de columnas creadas
cols_show = [c for c in df_m.columns if c.endswith('_l1p')][:6]
print("\nColumnas l1p creadas (ejemplo):", cols_show)
df_m[montos + cols_show].sample(5, random_state=7)



==== TotPptos ====


,original,log1p
min,0.0,0.000000
p50,638500.0,13.366879
p90,1678280.0,14.333281
p95,2214000.0,14.610312
p99,4198584.0,15.250258



==== TotPptosAbo ====


,original,log1p
min,0.0,0.000000
p50,19800.0,9.893488
p90,908552.0,13.719608
p95,1260840.0,14.047290
p99,1954173.2,14.485478



==== TotPptosAvan ====


,original,log1p
min,0.0,0.000000
p50,19500.0,9.878221
p90,900872.0,13.711120
p95,1252280.0,14.040477
p99,1935344.0,14.475796



Columnas l1p creadas (ejemplo): ['TotPptos_l1p', 'TotPptosAbo_l1p', 'TotPptosAvan_l1p', 'TicketPromPpto_l1p']


,TotPptos,TotPptosAbo,TotPptosAvan,TotPptos_l1p,TotPptosAbo_l1p,TotPptosAvan_l1p,TicketPromPpto_l1p
47581,491100.0,236600.0,236600.0,13.104405,12.374130,12.374130,12.411260
31266,677200.0,0.0,0.0,13.425723,0.000000,0.000000,12.732578
41379,862400.0,373800.0,373800.0,13.667476,12.831479,12.831479,12.568866
4054,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51708,622900.0,20900.0,20900.0,13.342143,9.947552,9.947552,12.648997


In [5]:
# 3. Cobertura de salud unificada
df_m = unificar_cobertura_salud(df_m)
df_m['CoberturaSalud'].value_counts(dropna=False).head(10)

CoberturaSalud
NaN                            23302
FONASA                         19076
Isapre                         10099
ISAPRE                          2470
CAJA COMPENSACION LOS ANDES      747
CAJA LOS HEROES                  382
KEEPSMILING CHILE SPA            239
Tarjeta Soy Providencia          224
DUOC UC                          128
CONSALUD                         101
Name: count, dtype: int64

In [6]:
# 4. Marcación de activos (≤ 730 días)
mask_act = marcar_activos(df_m, dias_umbral=730)
df_act = df_m[mask_act].copy()
df_inact = df_m[~mask_act].copy()
df_act.shape, df_inact.shape

((14141, 56), (43887, 56))

In [8]:
# 4.1 Sanity check de activos (corregido)

n_total = len(df_m)
n_act = len(df_act)
n_inact = len(df_inact)

print("Total:", n_total, " | Activos:", n_act, " | Inactivos:", n_inact)
print("Proporción activos:", round(n_act/n_total*100, 2), "%")

# Detectar columnas de atención (evitar las _pres)
ventanas = [c for c in df_m.columns if c.lower().startswith('atencion') and not c.endswith('_pres')]
print("Ventanas detectadas:", ventanas)

if ventanas:
    # Convertir columna por columna a numérico
    vnum = df_m[ventanas].apply(pd.to_numeric, errors='coerce')
    # Presencia por fila: ¿alguna ventana > 0?
    cualquier_atencion = vnum.fillna(0).gt(0).any(axis=1)
else:
    cualquier_atencion = pd.Series(False, index=df_m.index, name='cualquier_atencion')

# Regla oficial por días
dias = pd.to_numeric(df_m.get('DiasDesdeUltimaVisita'), errors='coerce')
por_dias = dias.le(730).fillna(False)

print("Activos por ventanas de atención:", int(cualquier_atencion.sum()))
print("Activos por días <= 730:", int(por_dias.sum()))

# Diferencia entre criterios (solo diagnóstico)
diff_rate = (por_dias.astype(int) - cualquier_atencion.astype(int)).abs().mean()
print("Diferencia relativa entre criterios:", round(100*diff_rate, 2), "%")




Total: 58028  | Activos: 14141  | Inactivos: 43887
Proporción activos: 24.37 %
Ventanas detectadas: ['Atencion15d', 'Atencion1m', 'Atencion2m', 'Atencion3m', 'Atencion6m', 'Atencion1a', 'Atencion2a', 'Atencion2am']
Activos por ventanas de atención: 14141
Activos por días <= 730: 14009
Diferencia relativa entre criterios: 0.23 %


In [11]:
# 5. Presencia de atención (ventanas 15d, 1m, 3m, 6m, 1a, 2a)

# Ventanas a usar (presencia 0/1)
ventanas_keep = ['Atencion15d','Atencion1m','Atencion3m','Atencion6m','Atencion1a','Atencion2a']

df_act = df_act.copy()

# Crear columnas *_pres
creadas = []
for v in ventanas_keep:
    if v in df_act.columns:
        # >0 => 1, NaN/<=0 => 0
        df_act[v + '_pres'] = (pd.to_numeric(df_act[v], errors='coerce') > 0).astype('uint8')
        creadas.append(v + '_pres')

print("Features de presencia creadas:", creadas)

# Cobertura por ventana (qué % de activos tiene ≥1 atención en esa ventana)
cov = df_act[creadas].mean().sort_values(ascending=False)
print("\nCobertura (proporción=1) por ventana seleccionada:\n", cov)

# (opcional) vistazo
df_act[['RutBeneficiario'] + creadas].head(3)


Features de presencia creadas: ['Atencion15d_pres', 'Atencion1m_pres', 'Atencion3m_pres', 'Atencion6m_pres', 'Atencion1a_pres', 'Atencion2a_pres']

Cobertura (proporción=1) por ventana seleccionada:
 Atencion2a_pres     1.000000
Atencion1a_pres     0.652429
Atencion6m_pres     0.444099
Atencion3m_pres     0.312920
Atencion1m_pres     0.171770
Atencion15d_pres    0.084152
dtype: float64


,RutBeneficiario,Atencion15d_pres,Atencion1m_pres,Atencion3m_pres,Atencion6m_pres,Atencion1a_pres,Atencion2a_pres
30475,13.840.435-8,0,0,0,0,0,1
30478,22.206.983-1,0,0,0,0,1,1
30480,17.322.720-5,0,0,0,0,0,1


In [12]:
# 5.1 Depurar presencia: eliminar redundantes (1a y 2a)
pres_cols_all = ['Atencion15d_pres','Atencion1m_pres','Atencion3m_pres','Atencion6m_pres',
                 'Atencion1a_pres','Atencion2a_pres']
keep_pres = ['Atencion15d_pres','Atencion1m_pres','Atencion3m_pres','Atencion6m_pres']

# Eliminar las redundantes si existen
drop_pres = [c for c in pres_cols_all if c not in keep_pres and c in df_act.columns]
df_act = df_act.drop(columns=drop_pres, errors='ignore')

print("Presencia mantenida:", [c for c in keep_pres if c in df_act.columns])
print("Eliminadas:", drop_pres)

# (opcional) sanity rápido
print("Cobertura final:\n", df_act[keep_pres].mean().sort_values(ascending=False).round(4))


Presencia mantenida: ['Atencion15d_pres', 'Atencion1m_pres', 'Atencion3m_pres', 'Atencion6m_pres']
Eliminadas: ['Atencion1a_pres', 'Atencion2a_pres']
Cobertura final:
 Atencion6m_pres     0.4441
Atencion3m_pres     0.3129
Atencion1m_pres     0.1718
Atencion15d_pres    0.0842
dtype: float64


In [14]:
# 6. Geografía para modelar — Top-20 a partir de TODA la base (df) + whitelist de comunas clave

def _clean_comuna(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    return s.replace({'nan':'Sin Comuna','None':'Sin Comuna','':'Sin Comuna'})

# 6.0 Limpieza estándar
df_all = df.copy()
df_all['Comuna_clean'] = _clean_comuna(df_all.get('Comuna', pd.Series(index=df_all.index)))
df_act = df_act.copy()
df_act['Comuna_clean'] = _clean_comuna(df_act.get('Comuna', pd.Series(index=df_act.index)))

# 6.1 Top-20 global (sobre TODA la base)
topN = 20
top_global = df_all['Comuna_clean'].value_counts().head(topN).index.tolist()

# 6.2 Whitelist (comunas que DEBEN quedar como categoría propia aunque no queden top en activos)
whitelist = {'Iquique'}  # agrega otras si negocio lo pide, p.ej.: {'Iquique','Providencia'}

# 6.3 Lista final de categorías propias
categorias_propias = sorted(set(top_global).union(whitelist).union({'Sin Comuna'}))

# 6.4 Aplicar mapeo al subset de activos
def map_comuna_grp(val: str) -> str:
    return val if val in categorias_propias else 'Otras/Infreq'

df_act['Comuna_grp'] = df_act['Comuna_clean'].map(map_comuna_grp)

# 6.5 Región limpia
if 'Region' in df_act.columns:
    df_act['Region'] = (df_act['Region'].astype(str).str.strip()
                        .replace({'nan':'Sin Región','None':'Sin Región','':'Sin Región'}))
else:
    df_act['Region'] = 'Sin Región'

# 6.6 Tabla para revisar
top_tab = (df_act['Comuna_grp'].value_counts()
           .to_frame('n')
           .assign(pct=lambda d: (d['n']/len(df_act)).round(4)))
print("Top categorías en Comuna_grp (activos, con top global + whitelist):")
display(top_tab.head(25))

# 6.7 Dummies
df_act_dum = pd.get_dummies(df_act, columns=['Comuna_grp','Region'], drop_first=False, dtype='uint8')
dum_cols = [c for c in df_act_dum.columns if c.startswith('Comuna_grp_') or c.startswith('Region_')]
print("Total dummies geográficas creadas:", len(dum_cols))
print("Ejemplo:", dum_cols[:12])


Top categorías en Comuna_grp (activos, con top global + whitelist):


,n,pct
Comuna_grp,,
Otras/Infreq,3582,0.2533
Santiago,2295,0.1623
Pudahuel,811,0.0574
Estación Central,810,0.0573
Maipú,758,0.0536
Cerro Navia,568,0.0402
Quinta Normal,536,0.0379
Puente Alto,512,0.0362
Renca,476,0.0337


Total dummies geográficas creadas: 35
Ejemplo: ['Comuna_grp_Cerro Navia', 'Comuna_grp_Conchalí', 'Comuna_grp_Estación Central', 'Comuna_grp_Iquique', 'Comuna_grp_La Florida', 'Comuna_grp_Lo Prado', 'Comuna_grp_Maipú', 'Comuna_grp_Otras/Infreq', 'Comuna_grp_Pedro Aguirre Cerda', 'Comuna_grp_Peñalolén', 'Comuna_grp_Providencia', 'Comuna_grp_Pudahuel']


In [18]:
# 6b. Empresa/Convenio (features)
import pandas as pd, unicodedata, re

def _norm_text_series(s: pd.Series) -> pd.Series:
    s = s.astype(str).fillna('').str.strip()
    def _norm_one(x: str) -> str:
        x = unicodedata.normalize('NFKD', x)
        x = ''.join(c for c in x if not unicodedata.combining(c))
        return x.upper()
    return s.apply(_norm_one)

df_act = df_act.copy()
col_emp = 'Empresa'
if col_emp not in df_act.columns:
    df_act[col_emp] = ''

# Normalizar texto
df_act['Empresa_clean'] = _norm_text_series(df_act[col_emp])
df_act['Empresa_clean'] = df_act['Empresa_clean'].replace({'NAN':'', 'NONE':'', 'NULL':'', '0':''})
df_act['Empresa_clean'] = df_act['Empresa_clean'].replace('', 'SIN EMPRESA')

# Flags: tiene empresa / es convenio
df_act['TieneEmpresa'] = (df_act['Empresa_clean'] != 'SIN EMPRESA').astype('uint8')
df_act['EsConvenio']   = df_act['Empresa_clean'].str.contains(r'CONV|CONVENIO', regex=True).astype('uint8')

# Agrupar empresas: Top-15 + SIN EMPRESA + OTRAS/INFREQ
top_k = 15
vc_emp = df_act['Empresa_clean'].value_counts()
top_emp = vc_emp[vc_emp.index != 'SIN EMPRESA'].head(top_k).index.tolist()

def _map_emp(x: str) -> str:
    if x == 'SIN EMPRESA':
        return 'SIN EMPRESA'
    return x if x in top_emp else 'OTRAS/INFREQ'

df_act['Empresa_grp'] = df_act['Empresa_clean'].map(_map_emp)

# Tabla de control
emp_tab = (df_act['Empresa_grp'].value_counts()
           .to_frame('n')
           .assign(pct=lambda d: (d['n']/len(df_act)).round(4)))
print("Top Empresa_grp (activos):")
display(emp_tab.head(25))

# Incorporar a df_act_dum y hacer one-hot de Empresa_grp
df_act_dum = df_act_dum.copy()
for col in ['TieneEmpresa','EsConvenio','Empresa_grp']:
    df_act_dum[col] = df_act[col].values

df_act_dum = pd.get_dummies(df_act_dum, columns=['Empresa_grp'], drop_first=False, dtype='uint8')

dum_emp = [c for c in df_act_dum.columns if c.startswith('Empresa_grp_')]
print("Dummies Empresa creadas:", len(dum_emp))
print("Ejemplos:", dum_emp[:10])


Top Empresa_grp (activos):


,n,pct
Empresa_grp,,
FONASA,10341,0.7313
ISAPRE,1812,0.1281
CAJA COMPENSACION LOS ANDES,665,0.0470
CAJA LOS HEROES,293,0.0207
OTRAS/INFREQ,244,0.0173
TARJETA SOY PROVIDENCIA,147,0.0104
KEEPSMILING CHILE SPA,114,0.0081
DUOC UC,111,0.0078
SIN EMPRESA,105,0.0074


Dummies Empresa creadas: 17
Ejemplos: ['Empresa_grp_BANCO DE CHILE', 'Empresa_grp_CAJA 18 DE SEPTIEMBRE TRABAJADORES', 'Empresa_grp_CAJA COMPENSACION LOS ANDES', 'Empresa_grp_CAJA LOS HEROES', 'Empresa_grp_CORP.IP. ESC. CONTADORES Y AUDITORE', 'Empresa_grp_DUOC UC', 'Empresa_grp_FONASA', 'Empresa_grp_HOSPITAL DR. LUIS CALVO MACKENNA', 'Empresa_grp_INST CHILENO BRITANICO DE CULTURA', 'Empresa_grp_ISAPRE']


In [19]:
dum_comuna = [c for c in df_act_dum.columns if c.startswith('Comuna_grp_')]
dum_region = [c for c in df_act_dum.columns if c.startswith('Region_')]
print("Dummies Comuna_grp:", len(dum_comuna))
print("Dummies Region:", len(dum_region))
print("Total dummies geográficas:", len(dum_comuna) + len(dum_region))


Dummies Comuna_grp: 21
Dummies Region: 14
Total dummies geográficas: 35


In [24]:
# 7 (v2). Ensamble del dataset de modelado con Empresa/Convenio (definitivo)

df_model = df_act_dum.copy()

# Numéricas (todas deben existir; las que no, se filtran abajo)
num_keep = [
    'Edad',
    'CantPptos','CantPptosAbo','CantPptosAvan',
    'TotPptos_l1p','TotPptosAbo_l1p','TotPptosAvan_l1p',
    'PctCumplimiento','TicketPromPpto_l1p'
]

# Presencias
pres_keep = [c for c in ['Atencion15d_pres','Atencion1m_pres','Atencion3m_pres','Atencion6m_pres'] if c in df_model.columns]

# Flags empresa/convenio
bin_keep = [c for c in ['TieneEmpresa','EsConvenio'] if c in df_model.columns]

# Dummies geográficas y de empresa
dum_keep = [c for c in df_model.columns
            if c.startswith('Comuna_grp_') or c.startswith('Region_') or c.startswith('Empresa_grp_')]

# Robustez: dejar solo existentes y fijar orden reproducible
num_keep = [c for c in num_keep if c in df_model.columns]
cols_final = num_keep + pres_keep + bin_keep + sorted(dum_keep)

# Construir X_v2
X_v2 = df_model[cols_final].copy()

# Tipos y NaN
if num_keep:
    X_v2[num_keep] = X_v2[num_keep].apply(pd.to_numeric, errors='coerce')

X_v2 = X_v2.fillna(0)

for c in pres_keep + bin_keep + sorted(dum_keep):
    if c in X_v2.columns:
        X_v2[c] = X_v2[c].astype('uint8')

for c in num_keep:
    if c in X_v2.columns:
        X_v2[c] = pd.to_numeric(X_v2[c], downcast='float')  # float32

print("Filas, columnas de X_v2:", X_v2.shape)
print("NaNs totales en X_v2:", int(X_v2.isna().sum().sum()))
print("Grupos -> num:", len(num_keep), "| pres:", len(pres_keep), "| bin:", len(bin_keep), "| dummies:", len(dum_keep))
print("Ejemplos de columnas:", X_v2.columns[:12].tolist())

# Export (X_v2) — usar ROOT para no quedar en notebooks/
out_path = ROOT / "data" / "processed" / "activos_for_model_v2.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
X_v2.to_csv(out_path, index=False, encoding="utf-8")
print("Exportado a:", out_path.resolve())


Filas, columnas de X_v2: (14141, 67)
NaNs totales en X_v2: 0
Grupos -> num: 9 | pres: 4 | bin: 2 | dummies: 52
Ejemplos de columnas: ['Edad', 'CantPptos', 'CantPptosAbo', 'CantPptosAvan', 'TotPptos_l1p', 'TotPptosAbo_l1p', 'TotPptosAvan_l1p', 'PctCumplimiento', 'TicketPromPpto_l1p', 'Atencion15d_pres', 'Atencion1m_pres', 'Atencion3m_pres']
Exportado a: /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/data/processed/activos_for_model_v2.csv


In [25]:
# 8 (v2). IDs para reenganchar clusters + trazabilidad (Titular y #beneficiarios)

# Base de IDs del set activo
ids_v2 = (df_act[['RutBeneficiario']]
          .copy().reset_index(drop=True)
          .rename_axis('idx_model')
          .reset_index())

# Enriquecimiento desde Clientes (si existe archivo)
CLI_PATH = ROOT / "data" / "raw" / "Tab_Clientes(2).csv"
if CLI_PATH.exists():
    dfc = pd.read_csv(CLI_PATH, low_memory=False)

    if {'RutBeneficiario','RutTitular'}.issubset(dfc.columns):
        # Map RutBeneficiario -> RutTitular (1a coincidencia por beneficiario)
        map_titular = (dfc[['RutBeneficiario','RutTitular']]
                       .dropna(subset=['RutBeneficiario'])
                       .drop_duplicates(subset=['RutBeneficiario'])
                       .set_index('RutBeneficiario')['RutTitular'])

        # Conteo de beneficiarios por titular
        ben_por_titular = (dfc.groupby('RutTitular')['RutBeneficiario']
                             .nunique()
                             .rename('Beneficiarios_por_titular'))

        # Enriquecer IDs
        ids_v2['RutTitular'] = ids_v2['RutBeneficiario'].map(map_titular)
        ids_v2 = ids_v2.merge(ben_por_titular, how='left', left_on='RutTitular', right_index=True)
    else:
        print("Aviso: Clientes no tiene RutBeneficiario y RutTitular; exporto IDs sin enriquecimiento.")
else:
    print(f"Aviso: no encuentro {CLI_PATH}; exporto IDs sin enriquecimiento.")

# Export
ids_out = ROOT / "data" / "processed" / "activos_ids_v2_plus.csv"
ids_out.parent.mkdir(parents=True, exist_ok=True)
ids_v2.to_csv(ids_out, index=False, encoding="utf-8")
print("IDs v2 PLUS exportados a:", ids_out.resolve())





IDs v2 PLUS exportados a: /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/data/processed/activos_ids_v2_plus.csv
